## PHASE 4: AI & ML (Days 12-14)

### **DAY 12 (20/01/26) – MLflow Basics**

### Learn:

- MLflow components (tracking, registry, models)
- Experiment tracking
- Model logging
- MLflow UI

### 🛠️ Tasks:

1. Train simple regression model
2. Log parameters, metrics, model
3. View in MLflow UI
4. Compare runs

In [0]:
silver = spark.sql("select * from ecommerce.default.ecommerce_full_data")

In [0]:
from pyspark.sql import functions as F
gold_ecommerce_full_data =  silver.groupBy('product_id')\
                .agg(
                    F.countDistinct(F.when(F.col('event_type')=='view',F.col('user_id'))).alias('views')\
                    ,F.countDistinct(F.when(F.col('event_type')=='purchase',F.col('user_id'))).alias('purchases')\
                    ,F.countDistinct(F.when(F.col('event_type')=='cart',F.col('user_id'))).alias('cart')
                    )\
                .filter(F.col('views')>0)\
                .withColumn('Conversion_rate',F.col('purchases')/F.col('views')*100)
gold_ecommerce_full_data.write.format('delta').saveAsTable('ecommerce.default.gold_ecommerce_full_data')

# Prepare data

In [0]:
# Load data
gold_df=spark.table('ecommerce.default.gold_ecommerce_full_data').toPandas()
gold_df.head()

In [0]:
print('Records =',gold_df.shape[0])
gold_df.isnull().sum()

In [0]:
gold_df[gold_df['Conversion_rate']!=0].shape[0]

In [0]:
from sklearn.model_selection import train_test_split
X = gold_df[["views", "cart"]]
y = gold_df["purchases"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

print("Train data records :",X_train.shape[0])
print("Test data records :",X_test.shape[0])

In [0]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import warnings
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

In [0]:
models = {
    "linear": LinearRegression(),
    "decision_tree": DecisionTreeRegressor(max_depth=5),
    "random_forest": RandomForestRegressor(n_estimators=100)
}

In [0]:
# Multiple models without autolog
mlflow.set_experiment('/Day_12_Experiments')
mlflow.end_run()
for name, model in models.items():
    with mlflow.start_run(run_name=f"{name}_model"):
        mlflow.log_param("model_type", name)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        mlflow.log_metric("r2_score", score)
        mlflow.sklearn.log_model(model, "model",input_example=X_train.iloc[:1])
        print(f"{name}: R² = {score:.4f}")

In [0]:
# Multiple models with autolog

mlflow.sklearn.autolog()
mlflow.set_experiment('/Day_12_Experiments')
mlflow.end_run()
for name, model in models.items():
    with mlflow.start_run(run_name=f"{name}_model_v2"):
        mlflow.log_param("model_type", name)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        mlflow.log_metric("r2_score", score)
        mlflow.sklearn.log_model(model, "model",input_example=X_train.iloc[:1])
        print(f"{name}: R² = {score:.4f}")